<a target="_parent" href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/amazon/navigator_tabular_aws_bedrock.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Gretel Navigator Tabular on Amazon Bedrock

This Notebook will walk you through deploying Gretel Navigator Tabular as a Bedrock Marketplace Model. You can deploy Gretel Navigator as an endpoint in Bedrock and interact with the model using the Gretel SDK.

This Notebook will walk you through the following steps:

* Deploy Gretel Navigator Tabular on AWS Bedrock
* Install and configure the Gretel SDK
* Generate synthetic data with the Gretel SDK and the Bedrock Endpoint
* Edit and augment existing data with the Gretel SDK and the Bedrock Endpoint

# Deploy Gretel Navigator

To get started, visit the [AWS Bedrock homepage](https://us-west-2.console.aws.amazon.com/bedrock/home?region=us-west-2#/) in the AWS Console. In this example we'll be using `us-west-2`.

1. Under **Foundation Models**, select **Model Catalog**:

<img src="https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/navigator_bedrock/1_model-catalog.png" alt="Model Catalog" width="70%">

2. Under **Providers** on the left side, select **Gretel**:

<img src="https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/navigator_bedrock/2_providers.png" alt="Provider" width="70%">

4. Click on **View subscription options**:

<img src="https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/navigator_bedrock/3_subscription-options.png" alt="Subscription Options" width="70%">


6. Click on **Subscribe**:

<img src="https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/navigator_bedrock/4_subscribe.png" alt="Subscribe" width="60%">


8. Wait for the subscription to complete:

<img src="https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/navigator_bedrock/5_subscription_complete.png" alt="Subscription Complete" width="70%">


10. Once the subscription is complete, click **Deploy**:

<img src="https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/navigator_bedrock/6_deploy.png" alt="Deploy" width="70%">


12. You should reach a configuration screen like below. For this example, we will use the defaults. Update the fields for your use case and modify the **Advanced Settings** as required.


When you are done with the configuration, click the **Deploy** button on the bottom right.

<img src="https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/navigator_bedrock/7_config_deploy.png" alt="Configure and Deploy" width="70%">


8. Remain on the page, and you should eventually see something like this:

<img src="https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/navigator_bedrock/8_in_progress.png" alt="Deployment Progress" width="70%">


Wait for the model to deploy and the **Endpoint status** to change from **Creating** to **In Service**

# Setup

From the **Marketplace deployments** page (see above). Retrieve the **Endpoint Name (ARN)** and set the variable below.

In [ ]:
# Install the Gretel SDK and boto3 (if you do not already have it)

!pip install -U -qq gretel-client boto3

In [ ]:
# Import required libraries
import boto3
import os
from getpass import getpass

In [ ]:
# If you'd prefer to use a profile, uncomment these lines and comment out the getpass cell below

# AWS Metadata - set these based on your deployment and AWS account
# ENDPOINT_NAME = "arn:aws:sagemaker:us-east-2:298655852039:endpoint/endpoint-quick-start-uvzqj"
## AWS_PROFILE = "marketplace-ops.poweruser"

# AWS_REGION = "us-west-2"

In [ ]:
# Set region and get credentials securely
default_region = "us-east-1"
AWS_ACCESS_KEY = getpass("AWS Access key: ")
AWS_SECRET_KEY = getpass("AWS Secret key: ")
AWS_SESSION_TOKEN = getpass("AWS Session token (press Enter if not using temporary credentials): ") or None
AWS_REGION = input(f"AWS Region [default: {default_region}]: ") or default_region
ENDPOINT_NAME = input("Endpoint name: ")

# Initialize the boto3 client with explicit credentials
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    aws_session_token=AWS_SESSION_TOKEN if AWS_SESSION_TOKEN else None
)

In [ ]:
# Create a Bedrock Adapter using the Gretel SDK

from gretel_client import Gretel

bedrock = Gretel.create_navigator_bedrock_adapter(bedrock_client, ENDPOINT_NAME)

# Generate and Augment Datasets using Gretel Navigator

Alright, we're now ready to start creating data! We'll first generate some data using a single prompt, and then we'll add a couple of new columns. Try out some of your own prompts to see how it works.

In [ ]:
# First we'll generate some data from only a prompt. We provide a prompt and some existing sample data to guide the generation process.

import pandas as pd

PROMPT = """Generate a mock dataset for users from the Foo company based in France.
Each user should have the following columns:
* first_name: traditional French first names.
* last_name: traditional French surnames.
* email: formatted as the first letter of their first name followed by their last name @foo.io (e.g., jdupont@foo.io)
* gender: Male/Female
* city: a city in France
* country: always 'France'.
"""

table_headers = ["first_name", "last_name", "email", "gender", "city", "country"]
table_data = [
    {
        "first_name": "Lea",
        "last_name": "Martin",
        "email": "lmartin@foo.io",
        "gender": "Female",
        "city": "Lyon",
        "country": "France",
    }
]

SAMPLE_DATA = pd.DataFrame(table_data, columns=table_headers)

metadata, synthetic_df = bedrock.generate(
    "gretelai/auto",
    PROMPT,
    num_records=10,
    sample_data=SAMPLE_DATA,
)

print(synthetic_df)
print("*******")
print(metadata)


In [ ]:
# Finally, we'll demonstrate Navigator's edit mode, which can augment existing datasets. In this example we'll take our previously
# generated Synthetic DF and ask Navigator to augment it with new columns.

EDIT_PROMPT = """Edit the table and add the following columns:
* occupation: a random occupation
* education level: make it relevant to the occupation
"""

metadata, augmented_df = bedrock.edit(
    "gretelai/auto",
    EDIT_PROMPT,
    seed_data=synthetic_df
)

print(augmented_df)
print("*******")
print(metadata)